# CLEAR

Chargemaster Location-based Exploration for Affordability & Reform

This is the notebook file primarily responsible for pre-processing data, attaching important information, and generating database files for the github page. Below you can find all information about how data is processed from the downloaded `.csv` files found on most hospital sites. This is an exploratory project focused on creating interactive visualzations and tools to better inform people about their healthcare. The repo can always be maintained by downloading the most current year data for the specific hospital and putting it through the scripts. It should be noted that this is NOT a comprehensive list, but it can potentially be scaled to a full working-standalone site with enough time. 

All pre-processing code is written in python. See the `.html` files for how the D3 visualizations work. 

## How it works (Copied from README)

Hospitals that have been added to this 'web-app' are stored in a `.csv` file for quick look up and ease of access. This points to the loc of it's Charge Master `.parquet` file which is then queried for the specific proceedure. Hospitals are gathered from the CSV list based on a radius look-up provided by the user. If a hospital in the radius does not offer the service, it will not display the price point compared to others in the radius. 

## List of Hospitals

These are the hospital's which data has been gathered and processed for thus far:

| State    | Hospital Name                     | Zipcode     | Date                 | File Size    | Link                                                            |
|----------|--------------------------------|-------------|-------------------|-------------|------------------------------------------------|
| NC        | Duke University Hospital     |     27710    |      09/2025      |   3.32 GB   |                                                                   |
| NC        | Wake Med                           |                   |                          |                   |                                                                 |
| NC        | REX UNC                             |                   |                          |                   |                                                                 |

## Outside Sources Used

- zip_centroids.csv courtesy of SimpleMaps data


***

## Data Processing

CSV files are too large to store on github, thus they are downloaded locally, converted to the necessary format, then uploaded. If you want to perform conversions yourself you will need to find the specific hospital chargemaster and document in the notebook accordingly.

Not all Charge Masters (CM) are formatted the same, as such, to keep this notebook from growing too large, custom python scripts will be made for unique CM's. This matters beccause some hospitals are regional or statewide 'chains' but can vary prices between locations. For example, 

**AdventHealth**
- AdventHealth Orlando
- AdventHealth Tampa
- AdventHealth Hendersonville

all are AdventHealth hospitals, but their prices and available procedures vary per location. However, the same script to clean and process their CM's works because the file structure doesn't change from loc to loc. Normally CM structure only changes from hospital to hospital (brand-wise), but I haven't looked at the majority of US hospitals so this statement might need to be amended. 

Think of this file as more of a "**Controller**" for the cleaning, while the cleaning process is performed by imported functions. Subsections from here on are labeled by State, be sure to check which Hospitals are in each subsection before uploading data. 

## Data Structure

As the JS is grabbing the parquet files, we want the structure of each one to be similar even though it's not a centeral DB structure. This is more for performance/optimization and reducing the front-end coding needed as my JS is not as strong as my python tbh. 

#### Filenames for parquet:
- generated using a seeded ID function in this notebook, which updates the .csv file that holds the index information for the hospitals

In [ ]:
# hospitals.csv updater/editor
import hashlib
import requests
from geopy.geocoders import Nominatim
import pandas as pd
import time

geolocator = Nominatim(user_agent="CLEAR-geoapi-2025")
csv_file = 'docs/data/hospitals.csv'
df = pd.read_csv(csv_file)

# construct address for geocoding only (don't modify original data)
def construct_geocoding_address(row):
    # Build clean address from original components
    address = f"{row['address']}, {row['city']}, {row['state']} {row['zip']}"
    return address

# get lat/lon from address with increased timeout and retry/delay
def get_lat_lon(address, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            location = geolocator.geocode(address, timeout=5)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address} (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None, None

# generate short unique ID based on ['hospital'] + full composite address (base36, 8 chars)
def generate_short_id(row):
    full_address = construct_geocoding_address(row)
    unique_string = f"{row['name']}_{full_address}"
    hash_int = int(hashlib.md5(unique_string.encode()).hexdigest(), 16)
    short_id = base36encode(hash_int)[:8]
    return short_id

# base36 encoding for shorter IDs
def base36encode(number):
    chars = '0123456789abcdefghijklmnopqrstuvwxyz'
    if number == 0:
        return '0'
    result = ''
    while number > 0:
        number, i = divmod(number, 36)
        result = chars[i] + result
    return result

# Add lat/lon and short_id to dataframe, set parquet_path to be '/data/prices/['state']/['id'].parquet'
def update_dataframe(df):
    
    # Don't modify the address column - just use it for geocoding
    def lat_lon_with_delay(row):
        geocoding_address = construct_geocoding_address(row)
        lat, lon = get_lat_lon(geocoding_address)
        time.sleep(1)  # 1 second delay per request
        return pd.Series([lat, lon])
    
    df[['lat', 'lon']] = df.apply(lat_lon_with_delay, axis=1)
    df['id'] = df.apply(generate_short_id, axis=1)
    df['parquet_path'] = df.apply(lambda row: f"docs/data/hospitals/{row['state']}/{row['id']}.parquet", axis=1)
    df.to_csv(csv_file, index=False)
    return

update_dataframe(df)
